<a href="https://colab.research.google.com/github/Gaurav-Ramachandra/Sem5-BIS_Lab/blob/main/Expt7%2012-12%3A%20Optimisation%20via%20Gene%20Expression%20Algos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install sympy

In [ ]:
import random
import numpy as np
from sympy import sympify, lambdify

# Define a basic set of functions and terminals for the GEP algorithm
functions = ['+', '-', '*', '/']
terminals = ['x', 'y', '1', '2', '3']

# Example fitness function - the goal is to minimize the error between the generated expression and a target output
def fitness_function(individual, x_val, y_val):
    try:
        result = evaluate_expression(individual, x_val, y_val)
        target = x_val * y_val  # Example target function
        return abs(result - target)
    except ZeroDivisionError:
        return 1000  # A high but finite penalty

def evaluate_expression(individual, x_val, y_val):
    try:
        expr = sympify(individual)
        func = lambdify(['x', 'y'], expr)
        return func(x_val, y_val)
    except Exception as e:
        return float('inf')  # Handle various exceptions

def initialize_population(pop_size, max_depth):
    population = []
    for _ in range(pop_size):
        individual = generate_random_expression(max_depth)
        population.append(individual)
    return population

def generate_random_expression(max_depth):
    if max_depth == 0:
        return random.choice(terminals)
    else:
        func = random.choice(functions)
        left = generate_random_expression(max_depth - 1)
        right = generate_random_expression(max_depth - 1)
        return f"({left} {func} {right})"

def selection(population, fitness_scores):
    # Tournament selection
    tournament_size = 5
    selected_individuals = []
    for _ in range(2):  # Select two parents
        tournament_participants = random.sample(population, tournament_size)
        tournament_fitness = [fitness_function(ind, x_val, y_val) for ind in tournament_participants]
        best_index = tournament_fitness.index(min(tournament_fitness))
        selected_individuals.append(tournament_participants[best_index])
    return selected_individuals

def crossover(parent1, parent2):
    # Two-point crossover
    point1 = random.randint(1, len(parent1) - 2)
    point2 = random.randint(point1 + 1, len(parent1) - 1)
    offspring1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    offspring2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]
    return offspring1, offspring2

def mutation(individual, max_depth):
    point = random.randint(1, len(individual) - 1)
    mutated_part = generate_random_expression(max_depth - 1)
    return individual[:point] + mutated_part

def gene_expression_programming(pop_size, max_depth, max_generations, x_val, y_val):
    population = initialize_population(pop_size, max_depth)
    best_individual = None
    best_fitness = float('inf')
    final_generation = 0

    for generation in range(max_generations):
        fitness_scores = [fitness_function(individual, x_val, y_val) for individual in population]

        best_generation_fitness = min(fitness_scores)
        best_generation_individual = population[fitness_scores.index(best_generation_fitness)]

        if best_generation_fitness < best_fitness:
            best_fitness = best_generation_fitness
            best_individual = best_generation_individual
            final_generation = generation + 1

        if best_fitness == 0:  # Early stopping
            break

        parent1, parent2 = selection(population, fitness_scores)
        offspring1, offspring2 = crossover(parent1, parent2)

        if random.random() < 0.1:
            offspring1 = mutation(offspring1, max_depth)
        if random.random() < 0.1:
            offspring2 = mutation(offspring2, max_depth)

        population = population[:pop_size // 2] + [offspring1, offspring2]

    return best_individual, best_fitness, final_generation

# Get user inputs
pop_size = int(input("Enter population size: "))
max_depth = int(input("Enter maximum depth of expressions: "))
max_generations = int(input("Enter maximum number of generations: "))
x_val = float(input("Enter value for x (e.g., 1): "))
y_val = float(input("Enter value for y (e.g., 2): "))

# Run GEP algorithm
best_solution, best_fitness, final_generation = gene_expression_programming(pop_size, max_depth, max_generations, x_val, y_val)

# Output the best solution
print(f"Best Solution: {best_solution} | Fitness: {best_fitness} | Generation: {final_generation}")


Enter population size: 20
Enter maximum depth of expressions: 3
Enter maximum number of generations: 100
Enter value for x (e.g., 1): 2
Enter value for y (e.g., 2): 3
Best Solution: (((2 - x) + (y * x)) - ((1 - 1) * (y * 1))) | Fitness: 0.0 | Generation: 44
